In [94]:
import numpy as np
import xarray as xr
import pandas as pd
import datetime


from datetime import timedelta, datetime, date
import calendar

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import os
import glob



# LOAD first dataset

## ok these data are a pain because they come from monthly ingrid data that has made up the attributes. So some steps are specific of these data, you might need to change stuff for other data.



In [142]:
ds = xr.open_dataset('https://iridl.ldeo.columbia.edu/expert/SOURCES/.Models/.NMME/.COLA-RSMAS-CCSM4/.MONTHLY/.sst/S/(1%20Jan%201982)/(1%20Dec%202010)/RANGE/L/0.5/11.5/RANGE/Y/-5/5/RANGEEDGES/X/190/240/RANGEEDGES/dods',
                      decode_times=False)

In [143]:
ds

<xarray.Dataset>
Dimensions:  (L: 12, M: 10, S: 348, X: 51, Y: 11)
Coordinates:
  * L        (L) float32 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5 10.5 11.5
  * M        (M) float32 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0
  * S        (S) float32 264.0 265.0 266.0 267.0 ... 608.0 609.0 610.0 611.0
  * Y        (Y) float32 -5.0 -4.0 -3.0 -2.0 -1.0 0.0 1.0 2.0 3.0 4.0 5.0
  * X        (X) float32 190.0 191.0 192.0 193.0 ... 237.0 238.0 239.0 240.0
Data variables:
    sst      (S, L, M, Y, X) float32 ...
Attributes:
    Conventions:  IRIDL

In [144]:
ds.S.attrs

{'pointwidth': 0,
 'long_name': 'Forecast Start Time',
 'expires': 1596242520,
 'standard_name': 'forecast_reference_time',
 'defaultvalue': 726.0,
 'calendar': '360',
 'gridtype': 0,
 'units': 'months since 1960-01-01'}

In [145]:
ds.coords['T'] = ('S','L'), ds.S+ds.L
ds.T.attrs = {'pointwidth': 0,
 'long_name': 'Forecast Start Time',
 'expires': 1596242520,
 'standard_name': 'forecast_reference_time',
 'defaultvalue': 726.0,
 'calendar': '360',
 'gridtype': 0,
 'units': 'months since 1960-01-01'}

In [146]:
ds.T.attrs

{'pointwidth': 0,
 'long_name': 'Forecast Start Time',
 'expires': 1596242520,
 'standard_name': 'forecast_reference_time',
 'defaultvalue': 726.0,
 'calendar': '360',
 'gridtype': 0,
 'units': 'months since 1960-01-01'}

## trick to overwrite the issues with INGRID files

In [147]:
def fix_calendar(ds, timevar='T'):
    if ds[timevar].attrs['calendar'] == '360':
        ds[timevar].attrs['calendar'] = '360_day'
    return ds

ds = fix_calendar(ds, 'S')
ds = fix_calendar(ds, 'T')
ds = xr.decode_cf(ds)
ds


<xarray.Dataset>
Dimensions:  (L: 12, M: 10, S: 348, X: 51, Y: 11)
Coordinates:
  * L        (L) float32 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5 10.5 11.5
  * M        (M) float32 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0
  * S        (S) object 1982-01-01 00:00:00 ... 2010-12-01 00:00:00
  * Y        (Y) float32 -5.0 -4.0 -3.0 -2.0 -1.0 0.0 1.0 2.0 3.0 4.0 5.0
  * X        (X) float32 190.0 191.0 192.0 193.0 ... 237.0 238.0 239.0 240.0
    T        (S, L) object ...
Data variables:
    sst      (S, L, M, Y, X) float32 ...
Attributes:
    Conventions:  IRIDL

In [150]:
ds.coords['S'] = [np.datetime64(it) for it in ds.S.values ]


In [151]:
ds = ds.sst.mean(dim=['X','Y','M'])

# Second dataset

In [152]:
ds1 = xr.open_dataset('https://iridl.ldeo.columbia.edu/expert/SOURCES/.NOAA/.NCEP/.EMC/.CMB/.GLOBAL/.Reyn_SmithOIv2/.monthly/.sst/Y/-5/5/RANGEEDGES/X/190/240/RANGEEDGES/dods',
                     decode_times=False)
ds1

<xarray.Dataset>
Dimensions:  (T: 463, X: 50, Y: 10)
Coordinates:
  * T        (T) float32 262.5 263.5 264.5 265.5 ... 721.5 722.5 723.5 724.5
  * Y        (Y) float32 -4.5 -3.5 -2.5 -1.5 -0.5 0.5 1.5 2.5 3.5 4.5
  * X        (X) float32 190.5 191.5 192.5 193.5 ... 236.5 237.5 238.5 239.5
Data variables:
    sst      (T, Y, X) float32 ...
Attributes:
    Conventions:  IRIDL

In [153]:
ds1 = fix_calendar(ds1, 'T')
ds1 = xr.decode_cf(ds1)
ds1


<xarray.Dataset>
Dimensions:  (T: 463, X: 50, Y: 10)
Coordinates:
  * T        (T) object 1981-11-16 00:00:00 ... 2020-05-16 00:00:00
  * Y        (Y) float32 -4.5 -3.5 -2.5 -1.5 -0.5 0.5 1.5 2.5 3.5 4.5
  * X        (X) float32 190.5 191.5 192.5 193.5 ... 236.5 237.5 238.5 239.5
Data variables:
    sst      (T, Y, X) float32 ...
Attributes:
    Conventions:  IRIDL

# I Calculated T from S and L before transforming them in dates, but if you have other data you would create a timedelta64 array.



In [154]:
dsT = ds.stack(SL=('S','L'))
dsT

<xarray.DataArray 'sst' (SL: 4176)>
array([26.46026 , 26.897198, 27.534616, ..., 26.663513, 26.822168,
       27.146496], dtype=float32)
Coordinates:
    T        (SL) object 1982-01-16 00:00:00 ... 2011-11-16 00:00:00
  * SL       (SL) MultiIndex
  - S        (SL) datetime64[ns] 1982-01-01 1982-01-01 ... 2010-12-01 2010-12-01
  - L        (SL) float64 0.5 1.5 2.5 3.5 4.5 5.5 ... 6.5 7.5 8.5 9.5 10.5 11.5

In [155]:
dsT = dsT.swap_dims({'SL':'T'})
dsT

<xarray.DataArray 'sst' (T: 4176)>
array([26.46026 , 26.897198, 27.534616, ..., 26.663513, 26.822168,
       27.146496], dtype=float32)
Coordinates:
  * T        (T) object 1982-01-16 00:00:00 ... 2011-11-16 00:00:00
    SL       (T) object (Timestamp('1982-01-01 00:00:00'), 0.5) ... (Timestamp('2010-12-01 00:00:00'), 11.5)

In [156]:
dsT = dsT.unstack()
dsT

<xarray.DataArray 'sst' (T: 4176)>
array([26.46026 , 26.897198, 27.534616, ..., 26.663513, 26.822168,
       27.146496], dtype=float32)
Coordinates:
  * T        (T) object 1982-01-16 00:00:00 ... 2011-11-16 00:00:00
    SL       (T) object (Timestamp('1982-01-01 00:00:00'), 0.5) ... (Timestamp('2010-12-01 00:00:00'), 11.5)

# let's change all T to time

In [157]:
dsT = dsT.rename({'T':'time'})
ds1 = ds1.rename({'T':'time'})

# selectr dates that are in the observations

## I change them to np.datetime64 - note these are not eactly the same thing but being monthly values it's ok



In [158]:
dsT.coords['time'] = [np.datetime64(it) for it in dsT.time.values ]

In [159]:
ds1.coords['time'] = [np.datetime64(it) for it in ds1.time.values ]

In [160]:
dsT.time

<xarray.DataArray 'time' (time: 4176)>
array(['1982-01-16T00:00:00.000000000', '1982-02-16T00:00:00.000000000',
       '1982-03-16T00:00:00.000000000', ..., '2011-09-16T00:00:00.000000000',
       '2011-10-16T00:00:00.000000000', '2011-11-16T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1982-01-16 1982-02-16 ... 2011-11-16
    SL       (time) object (Timestamp('1982-01-01 00:00:00'), 0.5) ... (Timestamp('2010-12-01 00:00:00'), 11.5)

In [161]:
ds1.time

<xarray.DataArray 'time' (time: 463)>
array(['1981-11-16T00:00:00.000000000', '1981-12-16T00:00:00.000000000',
       '1982-01-16T00:00:00.000000000', ..., '2020-03-16T00:00:00.000000000',
       '2020-04-16T00:00:00.000000000', '2020-05-16T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1981-11-16 1981-12-16 ... 2020-05-16

# now you have the same time coordinate. but you can't select the values, because you have multiple dates "time" in the forecast data

## for example for 1990-03-16 I have


In [162]:
dstemp = dsT.sel(time=ds1.time[100].values)

In [168]:
y_SL = [y for (x,y) in dstemp.SL.values]
x_SL = [x for (x,y) in dstemp.SL.values]
L_SL = np.array(y_SL)
S_SL = np.array(x_SL)

In [169]:
S_SL

array([Timestamp('1989-04-01 00:00:00'), Timestamp('1989-05-01 00:00:00'),
       Timestamp('1989-06-01 00:00:00'), Timestamp('1989-07-01 00:00:00'),
       Timestamp('1989-08-01 00:00:00'), Timestamp('1989-09-01 00:00:00'),
       Timestamp('1989-10-01 00:00:00'), Timestamp('1989-11-01 00:00:00'),
       Timestamp('1989-12-01 00:00:00'), Timestamp('1990-01-01 00:00:00'),
       Timestamp('1990-02-01 00:00:00'), Timestamp('1990-03-01 00:00:00')],
      dtype=object)

In [170]:
y_SL

[11.5, 10.5, 9.5, 8.5, 7.5, 6.5, 5.5, 4.5, 3.5, 2.5, 1.5, 0.5]

# to get the values SL corresponding to each time

In [ ]:
   Y -5 5 RANGEEDGES
   X 190 240 RANGEEDGES
   [X Y]average
   [L S]REORDER
   S 12 splitstreamgrid
   exch
   S 12 splitstreamgrid
   [S2] correlate
